# Plate detection

In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [11]:
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import skimage
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath('')
MEDIA_DIR = os.path.abspath('/media/disk2/amaltsev/car_lp_generator')

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

MODEL_DIR = os.path.join(MEDIA_DIR, 'model')

In [12]:
class PlatesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = 'plates'
    NUM_CLASSES = 1 + 1  # background + plate
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    LEARNING_RATE = 0.0002
    STEPS_PER_EPOCH = 200
    VALIDATION_STEPS = 50

#     IMAGE_RESIZE_MODE = "pad64"
#     IMAGE_MIN_DIM = 448 # None
    IMAGE_MAX_DIM = 1024
#     IMAGE_MIN_SCALE = None

#     MEAN_PIXEL = np.array([123.7, 116.8, 103.9]) TODO

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    # Non-max suppression threshold to filter RPN proposals.
    RPN_NMS_THRESHOLD = 0.8  # 0.7
    # Ratios of anchors at each cell (width/height)
    RPN_ANCHOR_RATIOS = [0.5, 1,  2]

    # Number of ROIs per image to feed to classifier/mask heads
    TRAIN_ROIS_PER_IMAGE = 256

    MAX_GT_INSTANCES = 5
    DETECTION_MAX_INSTANCES = 5
    DETECTION_MIN_CONFIDENCE = 0.7
    DETECTION_NMS_THRESHOLD = 0.2  # 0.3


config = PlatesConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        5
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.2
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0002
LOSS_WEIGHTS                   {'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'rpn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [13]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [14]:
class InferenceConfig(PlatesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=MODEL_DIR)

In [15]:
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
# model_path = model.find_last()
model_path = os.path.join(MODEL_DIR, 'mask_rcnn_plates_0060.h5')

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /media/disk2/amaltsev/car_lp_generator/model/mask_rcnn_plates_0060.h5


# Plate removal

In [16]:
def extend_mask(mask):
    new_mask = mask.copy()
    n = mask.shape[0]
    m = mask.shape[1]
    for i in range(n):
        for j in range(m):
            if mask[i][j]:
                for ki in range(max(0, i - 4), min(n, i + 5)):
                    for kj in range(max(0, j - 4), min(m, j + 5)):
                        new_mask[ki][kj] = True
    return new_mask

In [17]:
def remove_object(img, mask):
    n = img.shape[0]
    m = img.shape[1]
    new_img = img.copy()
    for i in range(n):
        for j in range(m):
            if mask[i][j]:
                sum_neighbours = [0, 0, 0]
                num_neighbours = 0
                for ki in range(max(0, i - 2), min(n, i + 3)):
                    for kj in range(max(0, j - 2), min(m, j + 3)):
                        if not mask[ki][kj]:
                            sum_neighbours += new_img[ki][kj]
                            num_neighbours += 1

                new_img[i][j] = [s_i / num_neighbours for s_i in sum_neighbours]
                mask[i][j] = False
    return new_img

In [ ]:
img_dir = os.path.join(MEDIA_DIR, 'frames_full')
masks_dir = os.path.join(MEDIA_DIR, 'masks')
counter = 0
for img_name in sorted(os.listdir(masks_dir)):
    if img_name.startswith('.'):
        continue
    if os.path.exists(os.path.join(MEDIA_DIR, 'removed_plates', img_name)):
        continue
    img = skimage.io.imread(os.path.join(img_dir, img_name))

    counter += 1

    mask = skimage.io.imread(os.path.join(MEDIA_DIR, 'masks', img_name), as_gray=True)
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            mask[i][j] = (mask[i][j] == 0)
    
    mask = extend_mask(mask)

    removed = remove_object(img, mask)
    skimage.io.imsave(os.path.join(MEDIA_DIR, 'removed_plates', img_name), removed)

In [9]:
img_dir = os.path.join(MEDIA_DIR, 'frames_full')
counter = 0
for img_name in sorted(os.listdir(img_dir)):
    if img_name.startswith('.'):
        continue
#     if os.path.exists(os.path.join(MEDIA_DIR, 'removed_plates', img_name)):
#         continue
    img = skimage.io.imread(os.path.join(img_dir, img_name))
    results = model.detect([img], verbose=1)

    if len(results) > 0 and len(results[0]['rois']) > 0:
        counter += 1
        white_img = np.ones([img.shape[0], img.shape[1], 3])
        white_img[:, :, :] *= 255
        mask = results[0]['masks'][:, :, 0]
        
        mask = extend_mask(mask)

        mask_img = visualize.apply_mask(white_img, mask, color=(0, 0, 0), alpha = 1)
        removed = remove_object(img, mask)
        # skimage.io.imsave(os.path.join(MEDIA_DIR, 'masks', img_name), mask_img)
        skimage.io.imsave(os.path.join(MEDIA_DIR, 'removed_plates', img_name), removed)

NameError: name 'results' is not defined

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

# sys.argv = ['-h', '-g 2','-v 2.2.0']
# from number_train import plate_loss

def plate_loss(y_true, y_pred):
    n = 9
    y_pred = tf.transpose(y_pred, perm=[1, 0, 2])
    y_true = tf.transpose(y_true, perm=[1, 0, 2])
    losses = [tf.keras.losses.categorical_crossentropy(y_true[i], y_pred[i]) / n for i in range(n)]
    total_loss = tf.math.reduce_sum(tf.convert_to_tensor(losses))
    return total_loss

def plate_acc(y_true, y_pred):
    n = 9
    y_pred = tf.transpose(y_pred, perm=[1, 0, 2])
    y_true = tf.transpose(y_true, perm=[1, 0, 2])
    acc_fn = tf.keras.metrics.CategoricalAccuracy()
    acc = [acc_fn(y_true[i], y_pred[i]) / n for i in range(n)]
    total_acc = tf.math.reduce_sum(tf.convert_to_tensor(acc))
    return total_acc


# config = tf.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4)
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)


img_dir = os.path.join(MEDIA_DIR, 'plates')

model_text = load_model('/media/disk2/amaltsev/car_lp_generator/model/fs_adam_m2_dense_256_drop_03.h5', custom_objects={'plate_loss': plate_loss, 'plate_acc': plate_acc})
images = [[cv2.imread(os.path.join(img_dir, x)), x] for x in sorted(os.listdir(img_dir))]
images = [img_i for img_i in images if img_i[0] is not None]
imgs = [img[0] for img in images]
for i in range(len(imgs)):
    imgs[i] = cv2.resize(imgs[i], (160, 40))
imgs = np.reshape(imgs, (len(imgs),) + imgs[0].shape)

results = model_text.predict(imgs)

NameError: name 'os' is not defined

In [ ]:
symbols = '0123456789ABCEHKMOPTXY_'
for i in range(len(results)):
    text = ''
    for rj in results[i]:
        if 1 in rj:
            text += symbols[list(rj).index(1)]
        else:
            text += 'A'
    cv2.imwrite(os.path.join(MEDIA_DIR, 'plates_text', images[i][1].split('.')[0] + '_' + text), images[i][0])

In [ ]:
# img_dir = os.path.join(MEDIA_DIR, 'frames_full')
# skip_counter = 0
# for img_name in os.listdir(img_dir):
#     skip_counter += 1
#     if skip_counter < 370:
#         continue
#     img = skimage.io.imread(os.path.join(img_dir, img_name))
#     results = model.detect([img], verbose=1)
#     if len(results) > 0 and len(results[0]['rois']) > 0:
#         roi = results[0]['rois'][0]
#         plate = img[roi[0]:roi[2], roi[1]:roi[3], :]
#         skimage.io.imsave(os.path.join(MEDIA_DIR, 'plates', img_name), plate)

In [ ]:
        
original_image = skimage.io.imread('/media/disk2/amaltsev/car_lp_generator/car_6895_mod.jpg')
    
plt.imshow(original_image)

results = model.detect([original_image], verbose=1)

r = results[0]
print('rois:', r['rois'], 'masks:', r['masks'], 'class_ids:', r['class_ids'], 'scores:', r['scores'])
print(r['masks'].shape)
print(r['masks'][0:1].shape)

# visualize.display_instances(original_image, r['rois'][0:1], r['masks'][:, :, 0:1], r['class_ids'][0:1], 
#                             [None, 'plate'], r['scores'][0:1], ax=get_ax())

visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            [None, 'plate'], r['scores'], ax=get_ax())